In [ ]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
import numpy as np


def ReLU(x):
    return np.maximum(x, 0)


def D_ReLU(x):
    return np.diag((x >= 0).astype(float))


def SoftMax(x):
    tmp = np.exp(x - np.max(x))
    return tmp / np.sum(tmp)


def D_SoftMax(x):
    tmp = SoftMax(x)
    return -1 * np.outer(tmp, tmp) + np.diag(tmp)


def CategoricalCrossEntropy(y_pred, y_true):
    tmp = np.clip(y_pred, 1e-8, 1-1e-8)
    return np.mean(-1 * np.sum(y_true * np.log2(tmp) + (1 - y_true) * np.log2(1 - tmp), axis=1), axis=0)


def D_CategoricalCrossEntropy(y_pred, y_true):
    return y_pred - y_true
 

def L2_loss(y_pred, y_true):
    return np.mean(np.sum((y_pred - y_true)**2, axis=1), axis=0)


def D_L2_loss(y_pred, y_true):
    return 2 * (y_pred - y_true) / y_pred.shape[0]


class DenseLayer:

    def __init__(self, n_inputs, n_outputs, activation, D_activation):
        self.weights = np.random.randn(n_outputs, n_inputs) / np.sqrt(n_inputs * n_outputs)
        self.bias = np.random.randn(n_outputs) / np.sqrt(n_outputs)
        self.activation = activation
        self.D_activation = D_activation
        self.x = None
        self.D_y = None
        self.weights_error_lst = []
        self.bias_error_lst = []
        self.weights_error_rp = 0
        self.bias_error_rp = 0
    
    def forward(self, x):
        self.x = x
        z = np.dot(self.weights, x) + self.bias
        self.D_y = self.D_activation(z)
        y = self.activation(z)
        return y

    def backward(self, output_error):
        bias_error = np.dot(self.D_y.T, output_error)
        self.bias_error_lst.append(bias_error)
        weights_error = np.outer(bias_error, self.x)
        self.weights_error_lst.append(weights_error)
        input_error = np.dot(self.weights.T, bias_error)
        return input_error
    
    def update(self, learning_rate, adaption_rate):
        weights_error = np.mean(np.array(self.weights_error_lst), axis=0)
        self.weights_error_rp = np.clip(adaption_rate * self.weights_error_rp + (1 - adaption_rate) * weights_error**2, 1e-9, np.inf)
        self.weights -= learning_rate / (np.sqrt(self.weights_error_rp)) * weights_error
        self.weights_error_lst = []
        bias_error = np.mean(np.array(self.bias_error_lst), axis=0)
        self.bias_error_rp = np.clip(adaption_rate * self.bias_error_rp + (1 - adaption_rate) * bias_error**2, 1e-9, np.inf)
        self.bias -= learning_rate / (np.sqrt(self.bias_error_rp)) * bias_error
        self.bias_error_lst = []

    def reset(self):
        self.x = None
        self.D_y = None
        self.weights_error_lst = []
        self.bias_error_lst = []
        self.weights_error_rp = 0
        self.bias_error_rp = 0


class Network:

    def __init__(self):
        self.layer_lst = []

    def predict(self, X):
        y_pred_lst = []
        for x in X:
            y_pred_lst.append(self._forward(x))
        y_pred = np.array(y_pred_lst)
        return y_pred

    def _forward(self, x):
        y_pred = x
        for layer in self.layer_lst:
            y_pred = layer.forward(y_pred)
        return y_pred
    
    def _backward(self, y):
        y_pred = y
        for layer in reversed(self.layer_lst):
            y_pred = layer.backward(y_pred)
        return y_pred

    def _update(self, learning_rate, adaption_rate):
        for layer in self.layer_lst:
            layer.update(learning_rate, adaption_rate)

    def _reset(self):
        for layer in self.layer_lst:
            layer.reset()

    def fit(self, X_train, y_train, learning_rate=1e-1, adaption_rate=0.9, N_epochs=1000, N_batch=100):
        self._reset()
        for index_epoch in range(N_epochs):
            batch_indices = np.random.randint(0, X_train.shape[0] - 1, [N_batch])
            X_batch = X_train[batch_indices]
            y_batch = y_train[batch_indices]
            y_pred = np.full_like(y_batch, fill_value=np.nan)
            for index_batch, (x, y) in enumerate(zip(X_batch, y_batch)):
                y_pred[index_batch, :] = self._forward(x)
                self._backward(D_CategoricalCrossEntropy(y_pred[index_batch, :], y))
            self._update(learning_rate, adaption_rate)
            loss = CategoricalCrossEntropy(y_pred, y_batch)
            print("Epoch {}/{} Loss: {}".format(int(index_epoch + 1), N_epochs, loss))


N_epochs = 500
N_batch = 100
N_inputs = train_images.shape[1]
N_outputs = train_labels.shape[1]
N_hidden_layer = 2
learning_rate = 1e-3
adaption_rate = 0.9

network = Network()
network.layer_lst.append(DenseLayer(N_inputs, N_inputs, ReLU, D_ReLU))
network.layer_lst.extend([DenseLayer(N_inputs, N_inputs, ReLU, D_ReLU) for _ in range(N_hidden_layer)])
network.layer_lst.append(DenseLayer(N_inputs, N_outputs, SoftMax, D_SoftMax))
network.fit(train_images, train_labels, learning_rate, adaption_rate, N_epochs, N_batch)

In [ ]:
predicted_labels = network.predict(test_images)
accuracy = np.sum(np.argmax(predicted_labels, axis=1) == np.argmax(test_labels, axis=1)) / predicted_labels.shape[0] * 100
print("Accuracy: {:.1f}%".format(accuracy))